### Doing the calculations on transaction data manually to ensure they match what the app produces.

In [9]:
import pandas as pd
import yfinance as yf

df = pd.read_csv('Foolish_Transactions.csv')

# Delete rows with Type SELL
df = df[df['Type'] != 'SELL']

# Add an Amount coloumn
df['Amount'] = df['Price'] * df['Shares']

# For each row in Amount, multiple the Price by the Shares and store it in the Amount column
df['Amount'] = df['Price'] * df['Shares']
df['Amount'] = (df['Price'] * df['Shares']).round(2)

# Add Current Shares column
df['Current Shares'] = df['Shares']

# Sort by ticker
df = df.sort_values('Ticker')

# Update Current Shares for splits
mask = (df['Ticker'] == 'AVGO') & (df['Date'] < '2024-07-15')
df.loc[mask, 'Current Shares'] = df.loc[mask, 'Shares'] * 10

mask = (df['Ticker'] == 'LRCX') & (df['Date'] < '2024-10-03')
df.loc[mask, 'Current Shares'] = df.loc[mask, 'Shares'] * 10

mask = (df['Ticker'] == 'NVDA') & (df['Date'] < '2024-06-10')
df.loc[mask, 'Current Shares'] = df.loc[mask, 'Shares'] * 10

mask = (df['Ticker'] == 'SMCI') & (df['Date'] < '2024-10-01')
df.loc[mask, 'Current Shares'] = df.loc[mask, 'Shares'] * 10

# Get current prices and calculate Current Value
def get_current_price(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.history(period="1d")['Close'].iloc[-1]
    except:
        return 0

df['Current Price'] = df['Ticker'].apply(lambda x: get_current_price(x) if x != 'TOTAL' else 0)
df['Current Value'] = (df['Current Price'] * df['Current Shares']).round(2)

# Add total row
total_row = pd.DataFrame({'Ticker': ['TOTAL'], 'Type': [''], 'Date': [''], 'Price': [0], 'Shares': [0], 'Amount': [df['Amount'].sum()], 'Current Shares': [0], 'Current Price': [0], 'Current Value': [df['Current Value'].sum()]})
df = pd.concat([df, total_row], ignore_index=True)

df.to_csv('Foolish_Calculations.csv', index=False, mode='w')

df

,Ticker,Type,Date,Price,Shares,Amount,Current Shares,Current Price,Current Value
0,AAPL,BUY,2023-03-28,156.82,0.127538,20.00,0.127538,209.110001,26.67
1,AAPL,BUY,2023-01-09,130.08,0.153775,20.00,0.153775,209.110001,32.16
2,AAPL,BUY,2023-04-17,164.96,0.121237,20.00,0.121237,209.110001,25.35
3,AAPL,BUY,2024-07-15,235.69,4.243000,1000.03,4.243000,209.110001,887.25
4,AAPL,BUY,2024-04-29,174.49,5.731000,1000.00,5.731000,209.110001,1198.41
...,...,...,...,...,...,...,...,...,...
169,WING,BUY,2025-02-18,308.97,3.244000,1002.30,3.244000,317.660004,1030.49
170,WING,BUY,2024-11-18,325.49,3.845000,1251.51,3.845000,317.660004,1221.40
171,XYZ,BUY,2023-03-13,68.60,0.291566,20.00,0.291566,66.010002,19.25
172,ZG,BUY,2023-02-27,41.40,0.484782,20.07,0.484782,73.870003,35.81


### Finding XIRR
I imported the transactions into the APP. The App doesn't handle splits, so we're going to find the portfolio value without splits and validate the total value an invested value match. Then we're going to do XIRR calculations to make sure those match. After that, we can add dividends and splits.


In [10]:
# Import Foolish_Calculations.csv into a dataframe
df = pd.read_csv('Foolish_Calculations.csv')

# Add a new column, "CV w/out Splits" where the value is Shares * Current Price. Add sum at the bottomo of the column
df['CV w/out Splits'] = df['Shares'] * df['Current Price'] 
# Sum the values in the new column and plac it at the bottom of the column
df.loc[df['Ticker'] == 'TOTAL', 'CV w/out Splits'] = df['CV w/out Splits'].sum()
# Format the column with 2 decimal places
df['CV w/out Splits'] = df['CV w/out Splits'].round(2)

# Create a new column called Cash Flows; Fill each cell with the value from the Amount column and convert it to a negative number.
df['Cash Flows'] = df['Amount'] * -1
df.loc[df['Ticker'] == 'TOTAL', 'Cash Flows'] = None

# Calculate the XIRR. 
# For cash flow in the Cash Flow column, use the date in the Date column.
# Add one positive cash flow, with today's date and the value the same as the sum of the Current Value column.
# Place the XIRR value in the data frame below the row with the totals. 
# In the cell left of the XIRR value, place the text "XIRR" and format the value with 2 decimal places.

from datetime import datetime
import numpy as np
from scipy.optimize import fsolve

# Prepare data for XIRR calculation
df['Date'] = pd.to_datetime(df['Date'])
df['Cash Flows'] = df['Cash Flows'].astype(float)
df['Current Value'] = df['Current Value'].astype(float)
df['CV w/out Splits'] = df['CV w/out Splits'].astype(float)

# Get cash flows and dates (excluding TOTAL row)
cash_flows = df[df['Ticker'] != 'TOTAL']['Cash Flows'].tolist()
dates = df[df['Ticker'] != 'TOTAL']['Date'].tolist()

# Add final positive cash flow (current portfolio value) with today's date
total_current_value = df[df['Ticker'] == 'TOTAL']['Current Value'].iloc[0]
cash_flows.append(total_current_value)
dates.append(pd.to_datetime(datetime.now().date()))

# Calculate XIRR using scipy
def xirr_func(rate, cash_flows, dates):
    return sum([cf / (1 + rate) ** ((date - dates[0]).days / 365.0) for cf, date in zip(cash_flows, dates)])

try:
    xirr_rate = fsolve(xirr_func, 0.1, args=(cash_flows, dates))[0]
    xirr_percentage = xirr_rate * 100  # Convert to percentage
except:
    xirr_percentage = 0  # If calculation fails

# Create XIRR row
xirr_date = datetime.now().strftime('%Y-%m-%d')
xirr_row = pd.DataFrame({
    'Ticker': ['XIRR'], 
    'Type': [f"{round(xirr_percentage, 2)}%"],
    'Date': [xirr_date], 
    'Price': [0], 
    'Shares': [0], 
    'Amount': [0], 
    'Current Shares': [0], 
    'Current Price': [0], 
    'Current Value': [0], 
    'CV w/out Splits': [0], 
    'Cash Flows': [None]
})

df = pd.concat([df, xirr_row], ignore_index=True)

# Format columns
df['Current Value'] = df['Current Value'].round(2)
df['CV w/out Splits'] = df['CV w/out Splits'].round(2)
df['Cash Flows'] = df['Cash Flows'].round(2)

df.to_csv('Foolish_XIRR_Calculations.csv', index=False, mode='w')   

df

/var/folders/_6/5gbfzqhn6r9d9_h6tl8zktvc0000gs/T/ipykernel_40942/1078872523.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, xirr_row], ignore_index=True)


,Ticker,Type,Date,Price,Shares,Amount,Current Shares,Current Price,Current Value,CV w/out Splits,Cash Flows
0,AAPL,BUY,2023-03-28 00:00:00,156.82,0.127538,20.00,0.127538,209.110001,26.67,26.67,-20.00
1,AAPL,BUY,2023-01-09 00:00:00,130.08,0.153775,20.00,0.153775,209.110001,32.16,32.16,-20.00
2,AAPL,BUY,2023-04-17 00:00:00,164.96,0.121237,20.00,0.121237,209.110001,25.35,25.35,-20.00
3,AAPL,BUY,2024-07-15 00:00:00,235.69,4.243000,1000.03,4.243000,209.110001,887.25,887.25,-1000.03
4,AAPL,BUY,2024-04-29 00:00:00,174.49,5.731000,1000.00,5.731000,209.110001,1198.41,1198.41,-1000.00
...,...,...,...,...,...,...,...,...,...,...,...
170,WING,BUY,2024-11-18 00:00:00,325.49,3.845000,1251.51,3.845000,317.660004,1221.40,1221.40,-1251.51
171,XYZ,BUY,2023-03-13 00:00:00,68.60,0.291566,20.00,0.291566,66.010002,19.25,19.25,-20.00
172,ZG,BUY,2023-02-27 00:00:00,41.40,0.484782,20.07,0.484782,73.870003,35.81,35.81,-20.07
173,TOTAL,NaN,NaT,0.00,0.000000,84204.95,0.000000,0.000000,106295.31,101146.39,NaN


### For testing purposes, create a file with only buys.
Right now I just want to ensure buys are handled correctly. Note, Sells are already removed from the calculations.csv files above.


In [11]:
# Load the data from Foolish_Transactions.csv into a dataframe
df = pd.read_csv('Foolish_Transactions.csv')

# Remove any rows where Type is 'SELL'
df = df[df['Type'] != 'SELL']

# Write the data frame to Foolish_Buys.csv
df.to_csv('Foolish_Buys.csv', index=False, mode='w')

df




,Ticker,Type,Date,Price,Shares,Amount
0,AMZN,BUY,2023-01-04,85.04,0.235183,19.999962
1,AAPL,BUY,2023-01-09,130.08,0.153775,20.003052
2,BROS,BUY,2023-01-17,34.68,0.576701,19.999991
3,UBER,BUY,2023-01-23,30.88,0.647773,20.003230
4,HOOD,BUY,2023-01-30,10.24,1.953300,20.001300
...,...,...,...,...,...,...
169,CPNG,BUY,2025-06-24,29.10,34.366000,1000.050600
170,META,BUY,2025-06-30,741.02,1.355000,1004.082100
171,HOOD,BUY,2025-07-01,93.57,1.068000,99.932760
172,HOOD,BUY,2025-07-02,99.47,1.008000,100.265760
